In [1]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2
import torchvision
from torchvision import transforms
import albumentations
import albumentations.pytorch
from torch.utils.tensorboard import SummaryWriter
import datetime

In [2]:
writer = SummaryWriter('runs/')

%load_ext tensorboard

2021-10-18 23:00:32.383500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 23:00:32.383529: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
torch.manual_seed(121)
device = torch.device("cuda")
torch.cuda.manual_seed_all(121)

In [4]:
# Dataset
class Dataset(Dataset):
    def __init__(self, transform = None):
        super().__init__()
        
        self.img_dir = "/home/temp_1/kangsanha/malocclusion/data"
        self.filenames = os.listdir(self.img_dir)

        self.transform = transform

    def __getitem__(self, index):
        label = MakeLabel(self.filenames[index])
        img_path = os.path.join(self.img_dir, self.filenames[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        # albumentations 라이브러리를 사용할 것인데, PIL을 쓰면 변환이 안된다하여 cv2로 변환해준다.
        # img = np.asarray_chkfinite(Image.open(img_path))
        
        if img.dtype == np.uint8:
            img = img / 255.0
        
        #img = ToTensor(img)
        #print(type(img))
        #print(np.shape(img))
        #img = img.transpose((2,0,1)).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img)
            img = aug['image']
        #img = img.values()
        
        #  print(np.shape(img)) # 검사코드
        # input = {'data' : img, 'label' : label}
        
        return img, label
        

    def __len__(self):
        
        return len(self.filenames)

    
def MakeLabel(filenames):
    input_img = filenames
    if "O" in input_img:
        label = 1#label = F.one_hot(torch.tensor(0), 2)
    else : label = 0#label = F.one_hot(torch.tensor(1), 2)

    return label

def ToTensor(image):
    image = image.transpose((2,0,1)).astype(np.float32)
    image = torch.from_numpy(image)
    return image

In [8]:
# albumentations train

albumentations_train = albumentations.Compose([
    albumentations.Resize(height=256, width=256),
    albumentations.OneOf([
        albumentations.HorizontalFlip(p=0.8),
        albumentations.RandomRotate90(p=0.8),
        albumentations.VerticalFlip(p=0.8)
    ], p=1),
    albumentations.OneOf([
        albumentations.MotionBlur(p=0.75),
        albumentations.OpticalDistortion(p=0.75),
        
    ], p=1),
    albumentations.pytorch.ToTensorV2()
])

In [9]:
full_dataset = Dataset(transform = albumentations_train)

In [10]:
full_dataset[0]

(tensor([[[0.7320, 0.7461, 0.7312,  ..., 0.7276, 0.7368, 0.7276],
          [0.7308, 0.7458, 0.7327,  ..., 0.7217, 0.7313, 0.7217],
          [0.7305, 0.7482, 0.7273,  ..., 0.7173, 0.7251, 0.7173],
          ...,
          [0.3863, 0.3909, 0.3867,  ..., 0.6915, 0.6726, 0.6915],
          [0.3820, 0.3884, 0.3781,  ..., 0.6957, 0.6834, 0.6957],
          [0.3820, 0.3884, 0.3781,  ..., 0.6957, 0.6834, 0.6957]],
 
         [[0.5274, 0.5334, 0.5232,  ..., 0.5114, 0.5236, 0.5114],
          [0.5244, 0.5361, 0.5189,  ..., 0.5029, 0.5162, 0.5029],
          [0.5264, 0.5312, 0.5137,  ..., 0.4942, 0.5159, 0.4942],
          ...,
          [0.2545, 0.2582, 0.2714,  ..., 0.5507, 0.5363, 0.5507],
          [0.2532, 0.2570, 0.2692,  ..., 0.5589, 0.5452, 0.5589],
          [0.2532, 0.2570, 0.2692,  ..., 0.5589, 0.5452, 0.5589]],
 
         [[0.4205, 0.4320, 0.4040,  ..., 0.3877, 0.3839, 0.3877],
          [0.4161, 0.4318, 0.4010,  ..., 0.3772, 0.3800, 0.3772],
          [0.4210, 0.4237, 0.3957,  ...,

In [11]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_datset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [12]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_datset, batch_size=4, shuffle=False, num_workers=0)

In [13]:
# model(ResNet50)

import torchvision.models.resnet as resnet
import torch.optim as optim

conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False) # 마찬가지로 전부 사이즈 조정
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 32, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [14]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3], 2, True)

In [15]:
resnet50.cuda()

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 

In [22]:
# 정의
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.0001)
loss_func = nn.CrossEntropyLoss().to(device)

In [17]:
from torchsummary import summary
summary(resnet50, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,

In [18]:
mode = 'train'

In [16]:
dataiter = iter(train_loader)
image, labels = dataiter.next()

In [17]:
# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(image)
writer.add_image('inhovation_images', img_grid)

In [18]:
# 모델 구조 올리기
writer.add_graph(resnet50, image.cuda().float())
writer.close()

In [19]:
# 전이 학습
resnet50 = torchvision.models.resnet50(pretrained=True).to(device)

In [20]:
resnet50.fc = nn.Linear(resnet50.fc.in_features, 2).to(device)

In [21]:
summary(resnet50, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [29]:
from pylab import *

In [25]:
def images_to_probs(net, images):
    '''
    학습된 신경망과 이미지 목록으로부터 예측 결과 및 확률을 생성합니다
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]

In [23]:
def plot_classes_preds(net, images, labels):
    '''
    학습된 신경망과 배치로부터 가져온 이미지 / 라벨을 사용하여 matplotlib
    Figure를 생성합니다. 이는 신경망의 예측 결과 / 확률과 함께 정답을 보여주며,
    예측 결과가 맞았는지 여부에 따라 색을 다르게 표시합니다. "images_to_probs"
    함수를 사용합니다.
    '''
    preds, probs = images_to_probs(net, images)
    # 배치에서 이미지를 가져와 예측 결과 / 정답과 함께 표시(plot)합니다
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [23]:
# train
loss_arr = []
if mode == 'train':
    for epoch in range(0, 150):
        resnet50.train()
        loss_arr = []
        for batch, data in enumerate(train_loader, 0):
            inputs, labels = data # input data, label 분리
            inputs = inputs.to(device).float()
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = resnet50(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            loss_arr += [loss.item()]
             
            
            
       # writer.add_scalar('trainin loss', np.mean(loss_arr), epoch * len(train_loader)+ batch)
        #writer.add_figure('predictions vs. actuals', plot_classes_preds(resnet50, inputs, labels),
                         #gloabal_step = epoch * len(train_loader) + batch)
        print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, 100, batch, 135, np.mean(loss_arr)))   
        loss_arr = []
        
        with torch.no_grad():
            resnet50.eval()
            correct = 0
            total = 0
            test_loss = 0
            acc = []
            criterion= nn.CrossEntropyLoss().to(device)
            for batch, data in enumerate(test_loader, 0):
                inputs, labels = data # input data, label 분리
                inputs = inputs.to(device).float()
                labels = labels.to(device)
                output = resnet50(inputs)
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                test_loss += criterion(output, labels).item()
                acc.append(100 * correct/total)

                print('\nTest set : Average loss:{:.4f}, Accuracy: {}/{}({:.0f}%)\n'.format(
                                  test_loss, correct, total, 100 * correct/total
                              ))
print('Finish!!')

TRAIN: EPOCH 0000 / 0100 | BATCH 0134 / 0135 | LOSS 0.6686

Test set : Average loss:0.7127, Accuracy: 2/4(50%)


Test set : Average loss:1.5214, Accuracy: 4/8(50%)


Test set : Average loss:2.4845, Accuracy: 6/12(50%)


Test set : Average loss:2.9442, Accuracy: 9/16(56%)


Test set : Average loss:3.3372, Accuracy: 12/20(60%)


Test set : Average loss:4.0146, Accuracy: 14/24(58%)


Test set : Average loss:4.6099, Accuracy: 17/28(61%)


Test set : Average loss:5.3008, Accuracy: 19/32(59%)


Test set : Average loss:5.5628, Accuracy: 23/36(64%)


Test set : Average loss:6.5555, Accuracy: 25/40(62%)


Test set : Average loss:7.7122, Accuracy: 26/44(59%)


Test set : Average loss:8.3726, Accuracy: 29/48(60%)


Test set : Average loss:8.8025, Accuracy: 32/52(62%)


Test set : Average loss:9.8698, Accuracy: 34/56(61%)


Test set : Average loss:10.9309, Accuracy: 36/60(60%)


Test set : Average loss:12.1245, Accuracy: 37/64(58%)


Test set : Average loss:12.7920, Accuracy: 40/68(59%)


Test set


Test set : Average loss:2.4598, Accuracy: 23/28(82%)


Test set : Average loss:2.6159, Accuracy: 27/32(84%)


Test set : Average loss:2.6944, Accuracy: 31/36(86%)


Test set : Average loss:3.5750, Accuracy: 33/40(82%)


Test set : Average loss:3.8722, Accuracy: 37/44(84%)


Test set : Average loss:4.2985, Accuracy: 40/48(83%)


Test set : Average loss:4.5713, Accuracy: 44/52(85%)


Test set : Average loss:4.9561, Accuracy: 48/56(86%)


Test set : Average loss:5.7304, Accuracy: 51/60(85%)


Test set : Average loss:6.3297, Accuracy: 54/64(84%)


Test set : Average loss:6.5736, Accuracy: 58/68(85%)


Test set : Average loss:7.1794, Accuracy: 61/72(85%)


Test set : Average loss:7.6187, Accuracy: 65/76(86%)


Test set : Average loss:7.9693, Accuracy: 68/80(85%)


Test set : Average loss:8.2380, Accuracy: 72/84(86%)


Test set : Average loss:8.5405, Accuracy: 75/88(85%)


Test set : Average loss:8.7554, Accuracy: 79/92(86%)


Test set : Average loss:9.3252, Accuracy: 82/96(85%)


Test set 


Test set : Average loss:3.4024, Accuracy: 51/56(91%)


Test set : Average loss:3.7929, Accuracy: 55/60(92%)


Test set : Average loss:4.8610, Accuracy: 58/64(91%)


Test set : Average loss:5.2983, Accuracy: 61/68(90%)


Test set : Average loss:5.8994, Accuracy: 63/72(88%)


Test set : Average loss:6.5675, Accuracy: 65/76(86%)


Test set : Average loss:6.6446, Accuracy: 69/80(86%)


Test set : Average loss:6.6996, Accuracy: 73/84(87%)


Test set : Average loss:7.3106, Accuracy: 75/88(85%)


Test set : Average loss:7.3698, Accuracy: 79/92(86%)


Test set : Average loss:7.8347, Accuracy: 81/96(84%)


Test set : Average loss:7.9742, Accuracy: 85/100(85%)


Test set : Average loss:8.5186, Accuracy: 88/104(85%)


Test set : Average loss:8.7891, Accuracy: 91/108(84%)


Test set : Average loss:9.2130, Accuracy: 94/112(84%)


Test set : Average loss:9.2351, Accuracy: 98/116(84%)


Test set : Average loss:9.4051, Accuracy: 102/120(85%)


Test set : Average loss:9.4366, Accuracy: 106/124(85%)





Test set : Average loss:9.5617, Accuracy: 77/88(88%)


Test set : Average loss:9.7147, Accuracy: 81/92(88%)


Test set : Average loss:9.9759, Accuracy: 84/96(88%)


Test set : Average loss:10.2305, Accuracy: 87/100(87%)


Test set : Average loss:10.8272, Accuracy: 90/104(87%)


Test set : Average loss:10.9463, Accuracy: 94/108(87%)


Test set : Average loss:11.2364, Accuracy: 97/112(87%)


Test set : Average loss:11.5256, Accuracy: 100/116(86%)


Test set : Average loss:11.8138, Accuracy: 103/120(86%)


Test set : Average loss:12.0472, Accuracy: 106/124(85%)


Test set : Average loss:12.0745, Accuracy: 110/128(86%)


Test set : Average loss:12.3335, Accuracy: 113/132(86%)


Test set : Average loss:13.1559, Accuracy: 115/135(85%)

TRAIN: EPOCH 0013 / 0100 | BATCH 0134 / 0135 | LOSS 0.2197

Test set : Average loss:0.0423, Accuracy: 4/4(100%)


Test set : Average loss:0.2455, Accuracy: 8/8(100%)


Test set : Average loss:0.2528, Accuracy: 12/12(100%)


Test set : Average loss:0.4513, Acc


Test set : Average loss:16.1724, Accuracy: 95/116(82%)


Test set : Average loss:16.4675, Accuracy: 98/120(82%)


Test set : Average loss:16.4679, Accuracy: 102/124(82%)


Test set : Average loss:16.5232, Accuracy: 106/128(83%)


Test set : Average loss:18.2320, Accuracy: 109/132(83%)


Test set : Average loss:18.7435, Accuracy: 111/135(82%)

TRAIN: EPOCH 0017 / 0100 | BATCH 0134 / 0135 | LOSS 0.1408

Test set : Average loss:0.0373, Accuracy: 4/4(100%)


Test set : Average loss:1.3605, Accuracy: 7/8(88%)


Test set : Average loss:1.8614, Accuracy: 10/12(83%)


Test set : Average loss:2.1677, Accuracy: 13/16(81%)


Test set : Average loss:2.1957, Accuracy: 17/20(85%)


Test set : Average loss:2.1994, Accuracy: 21/24(88%)


Test set : Average loss:2.2505, Accuracy: 25/28(89%)


Test set : Average loss:2.3580, Accuracy: 29/32(91%)


Test set : Average loss:2.5914, Accuracy: 32/36(89%)


Test set : Average loss:4.7306, Accuracy: 34/40(85%)


Test set : Average loss:8.1076, Accuracy: 35/44

...


Test set : Average loss:0.0014, Accuracy: 4/4(100%)


Test set : Average loss:0.3488, Accuracy: 7/8(88%)


Test set : Average loss:0.3505, Accuracy: 11/12(92%)


Test set : Average loss:0.3511, Accuracy: 15/16(94%)


Test set : Average loss:0.3555, Accuracy: 19/20(95%)


Test set : Average loss:0.3560, Accuracy: 23/24(96%)


Test set : Average loss:0.3826, Accuracy: 27/28(96%)


Test set : Average loss:0.3827, Accuracy: 31/32(97%)


Test set : Average loss:0.3924, Accuracy: 35/36(97%)


Test set : Average loss:2.3025, Accuracy: 37/40(92%)


Test set : Average loss:2.7962, Accuracy: 40/44(91%)


Test set : Average loss:2.9499, Accuracy: 44/48(92%)


Test set : Average loss:3.1751, Accuracy: 47/52(90%)


Test set : Average loss:3.1765, Accuracy: 51/56(91%)


Test set : Average loss:5.4257, Accuracy: 54/60(90%)


Test set : Average loss:7.4030, Accuracy: 57/64(89%)


Test set : Average loss:7.8673, Accuracy: 60/68(88%)


Test set : Average loss:8.6519, Accuracy: 63/72(88%)


Test set : A


Test set : Average loss:0.2411, Accuracy: 32/32(100%)


Test set : Average loss:0.2531, Accuracy: 36/36(100%)


Test set : Average loss:1.1826, Accuracy: 37/40(92%)


Test set : Average loss:1.7020, Accuracy: 39/44(89%)


Test set : Average loss:1.8857, Accuracy: 43/48(90%)


Test set : Average loss:2.9971, Accuracy: 46/52(88%)


Test set : Average loss:3.1263, Accuracy: 50/56(89%)


Test set : Average loss:4.7355, Accuracy: 53/60(88%)


Test set : Average loss:8.2462, Accuracy: 56/64(88%)


Test set : Average loss:8.3348, Accuracy: 60/68(88%)


Test set : Average loss:8.8263, Accuracy: 63/72(88%)


Test set : Average loss:8.9332, Accuracy: 67/76(88%)


Test set : Average loss:8.9717, Accuracy: 71/80(89%)


Test set : Average loss:9.0184, Accuracy: 75/84(89%)


Test set : Average loss:9.4835, Accuracy: 78/88(89%)


Test set : Average loss:9.5312, Accuracy: 82/92(89%)


Test set : Average loss:10.2373, Accuracy: 85/96(89%)


Test set : Average loss:10.3803, Accuracy: 89/100(89%)


Test


Test set : Average loss:7.7648, Accuracy: 56/60(93%)


Test set : Average loss:9.9658, Accuracy: 59/64(92%)


Test set : Average loss:9.9707, Accuracy: 63/68(93%)


Test set : Average loss:9.9767, Accuracy: 67/72(93%)


Test set : Average loss:10.6191, Accuracy: 70/76(92%)


Test set : Average loss:10.6224, Accuracy: 74/80(92%)


Test set : Average loss:10.6229, Accuracy: 78/84(93%)


Test set : Average loss:11.4175, Accuracy: 81/88(92%)


Test set : Average loss:11.4322, Accuracy: 85/92(92%)


Test set : Average loss:11.7213, Accuracy: 88/96(92%)


Test set : Average loss:12.3424, Accuracy: 91/100(91%)


Test set : Average loss:12.3817, Accuracy: 95/104(91%)


Test set : Average loss:12.3951, Accuracy: 99/108(92%)


Test set : Average loss:12.8891, Accuracy: 102/112(91%)


Test set : Average loss:12.8924, Accuracy: 106/116(91%)


Test set : Average loss:12.8954, Accuracy: 110/120(92%)


Test set : Average loss:12.8954, Accuracy: 114/124(92%)


Test set : Average loss:12.9041, Accurac


Test set : Average loss:9.3169, Accuracy: 76/88(86%)


Test set : Average loss:9.3277, Accuracy: 80/92(87%)


Test set : Average loss:9.7453, Accuracy: 83/96(86%)


Test set : Average loss:9.7733, Accuracy: 87/100(87%)


Test set : Average loss:9.8080, Accuracy: 91/104(88%)


Test set : Average loss:9.8347, Accuracy: 95/108(88%)


Test set : Average loss:9.9946, Accuracy: 99/112(88%)


Test set : Average loss:9.9952, Accuracy: 103/116(89%)


Test set : Average loss:10.8755, Accuracy: 106/120(88%)


Test set : Average loss:10.8755, Accuracy: 110/124(89%)


Test set : Average loss:10.8799, Accuracy: 114/128(89%)


Test set : Average loss:13.1957, Accuracy: 117/132(89%)


Test set : Average loss:13.6357, Accuracy: 119/135(88%)



KeyboardInterrupt: 

In [26]:
torch.save(resnet50.state_dict(), 'model_statedict.pt')

In [27]:
torch.save(resnet50, 'model_whole.pt')

In [29]:
inputs, labels = enumerate(test_loader[0],1)

TypeError: 'DataLoader' object is not subscriptable

In [31]:
full_dataset[0]

(tensor([[[0.7367, 0.7444, 0.7448,  ..., 0.7272, 0.7207, 0.7089],
          [0.7256, 0.7554, 0.7375,  ..., 0.7280, 0.6968, 0.7109],
          [0.7287, 0.7481, 0.7355,  ..., 0.7163, 0.6828, 0.7028],
          ...,
          [0.3764, 0.3947, 0.3642,  ..., 0.7066, 0.6962, 0.7089],
          [0.3798, 0.3808, 0.3828,  ..., 0.6848, 0.6933, 0.7149],
          [0.3798, 0.3808, 0.3828,  ..., 0.6848, 0.6933, 0.7149]],
 
         [[0.5248, 0.5425, 0.5232,  ..., 0.5075, 0.5097, 0.4844],
          [0.5217, 0.5516, 0.5159,  ..., 0.5153, 0.4838, 0.4820],
          [0.5130, 0.5363, 0.5080,  ..., 0.4983, 0.4593, 0.5048],
          ...,
          [0.2676, 0.2718, 0.2381,  ..., 0.5699, 0.5729, 0.5756],
          [0.2708, 0.2696, 0.2695,  ..., 0.5480, 0.5617, 0.5848],
          [0.2708, 0.2696, 0.2695,  ..., 0.5480, 0.5617, 0.5848]],
 
         [[0.4065, 0.4379, 0.4213,  ..., 0.3802, 0.4043, 0.3726],
          [0.4015, 0.4470, 0.4100,  ..., 0.3953, 0.3745, 0.3749],
          [0.3954, 0.4305, 0.3905,  ...,